In [1]:
# Utils
import torch
import numpy as np

# ML models
from openxai.LoadModel import LoadModel

# Data loaders
import openxai.dataloader as return_loaders

# Explanation models
from openxai.Explainer import Explainer

# Evaluation methods
from.openxai.Evaluator import Evaluator

In [2]:
# Choose the model and the data set you wish to generate explanations for
data_loader_batch_size = 32
data_name = 'adult' 
model_name = 'ann'

### (0) Explanation method hyperparameters

In [3]:
# Hyperparameters for Lime
lime_mode = 'tabular'
lime_sample_around_instance = True
lime_kernel_width = 0.75
lime_n_samples = 1000
lime_discretize_continuous = False
lime_standard_deviation = float(np.sqrt(0.03))

### (1) Data Loaders

In [4]:
# Get training and test loaders
loader_train, loader_test = loaders.return_loaders(data_name=data_name,
                                                   download=True,
                                                   batch_size=data_loader_batch_size)
data_iter = iter(loader_test)
inputs, labels = data_iter.next()

In [5]:
# get full training data set
data_all = torch.FloatTensor(loader_train.dataset.data)

### (2) Load a pretrained ML model

In [6]:
# Load pretrained ml model
model = LoadModel(data_name=data_name,
                  ml_model=model_name,
                  pretrained=True)

### (3) Choose an explanation method

#### I: Explanation method with particular hyperparameters (LIME)

In [7]:
# You can supply your own set of hyperparameters like so:
param_dict_lime = dict()
param_dict_lime['dataset_tensor'] = data_all
param_dict_lime['std'] = lime_standard_deviation
param_dict_lime['mode'] = lime_mode
param_dict_lime['sample_around_instance'] = lime_sample_around_instance
param_dict_lime['kernel_width'] = lime_kernel_width
param_dict_lime['n_samples'] = lime_n_samples
param_dict_lime['discretize_continuous'] = lime_discretize_continuous
lime = Explainer(method='lime',
                 model=model,
                 dataset_tensor=data_all,
                 param_dict_lime=param_dict_lime)

In [8]:
lime_custom = lime.get_explanation(inputs, 
                                   label=labels)

100%|██████████████████████████████████████████████████████████████████████████████████| 32/32 [00:00<00:00, 88.74it/s]


In [9]:
lime_custom[0,:]

tensor([ 0.1547,  0.0862,  0.4036,  0.7915,  0.1726,  0.1953,  0.0136, -0.0265,
        -0.2166, -0.0555,  0.0358,  0.0211,  0.0024])

#### II: Explanation method with default hyperparameters (LIME)

In [10]:
# You can also use the default hyperparameters likes so:
lime = Explainer(method='lime',
                 model=model,
                 dataset_tensor=data_all,
                 param_dict_lime=None)
lime_default = lime.get_explanation(inputs.float(), 
                                    label=labels)
lime_default[0,:]

100%|██████████████████████████████████████████████████████████████████████████████████| 32/32 [00:00<00:00, 87.67it/s]


tensor([ 0.1497,  0.0841,  0.4288,  0.8380,  0.1662,  0.1876,  0.0294,  0.0013,
        -0.1977, -0.0070,  0.0115, -0.0198,  0.0198])

#### III: Explanation method with default hyperparameters (IG)

In [11]:
# To use a different explanation method change the method name like so
ig = Explainer(method='ig',
               model=model,
               dataset_tensor=data_all,
               param_dict_lime=None)
ig_default = ig.get_explanation(inputs.float(), 
                                label=labels)
ig_default[0,:]

tensor([-0.0361, -0.0216, -0.0885, -0.1858, -0.0361, -0.0432, -0.0047,  0.0008,
         0.0454,  0.0069, -0.0077,  0.0003, -0.0015], dtype=torch.float64)

### (4) Choose an evaluation metric

In [ ]:
evaluator = Evaluator(input_dict: dict,
                      inputs=inputs,
                      labels=labels, 
                      model=model, 
                      explainer=lime_default)

In [ ]:
# evaluate pairwise comparison
evaluator(metric='PRA')

In [ ]:
# evaluate rankcorrelation
evaluator(metric='RC')